# Test MISA CRM API Endpoints

Notebook này dùng để kiểm tra việc lấy dữ liệu từ các API endpoint của MISA CRM. 

**Yêu cầu:** Cần điền đầy đủ `MISA_CRM_CLIENT_ID` và `MISA_CRM_CLIENT_SECRET` trong file `.env` trước khi chạy.

In [1]:
import requests
import os
import json
import pandas as pd
from dotenv import load_dotenv

# Tải các biến môi trường từ file .env
load_dotenv()

# Lấy thông tin từ biến môi trường
MISA_CLIENT_ID = os.getenv('MISA_CRM_CLIENT_ID')
MISA_CLIENT_SECRET = os.getenv('MISA_CRM_CLIENT_SECRET')
BASE_URL = 'https://crmconnect.misa.vn'

print(f"Client ID: {MISA_CLIENT_ID}")
print(f"Client Secret is {'set' if MISA_CLIENT_SECRET else 'not set'}")

Client ID: FACOLOS
Client Secret is set


## 1. Lấy Access Token

In [2]:
def get_misa_access_token():
    """Hàm để lấy access token từ MISA CRM API."""
    token_url = f"{BASE_URL}/api/v2/Account"
    
    if not MISA_CLIENT_ID or not MISA_CLIENT_SECRET:
        print('Lỗi: Vui lòng điền MISA_CRM_CLIENT_ID và MISA_CRM_CLIENT_SECRET trong file .env')
        return None
    
    payload = {
        'client_id': MISA_CLIENT_ID,
        'client_secret': MISA_CLIENT_SECRET
    }
    headers = {'Content-Type': 'application/json'}
    
    try:
        response = requests.post(token_url, json=payload, headers=headers, timeout=30)
        response.raise_for_status()
        
        result = response.json()
        if result.get('success') and result.get('data'):
            access_token = result['data']
            print('Lấy access token thành công!')
            return access_token
        else:
            print(f"Lấy token thất bại: {result.get('error_message', result)}")
            return None
    except requests.exceptions.RequestException as e:
        print(f'Lỗi kết nối khi lấy token: {e}')
        return None

access_token = get_misa_access_token()
# In một phần token để xác nhận
if access_token:
    print(f'Access Token (Full): {access_token}')

Lấy access token thành công!
Access Token (Full): eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJwYXlsb2FkZGF0YSI6Ik9NdVc2dytKL0o4dWN0ZUVwQ2x4dkw0aHFzM2VzekVna3J6Yms3ZmxCUVZWYnBwTVZDczNtMWxRRC9DcGRHNnhJTnkrL2tnZ3N1Y3hjdWRwZU9rTDRENTJKakxIN1lRM2IyV0lKVW54ZnE3MGJJeWxrNVlsVVo4MTNwVDc3S2U3ZzF1WU9QUFM5bjJGUk5oNlREUE1kMUxDVmVvcnUvN1FyeDhyNUg1N2ZjYjlya2g2bzhnRE1aWllpdWZWQXU5MW4rbEh3N01YazFsd1RzRGhRWE1XaGxIL3hhK0U0ZUhLQ3EvZERMV2taclAySGs3bnFlNWdIb0lJa09BckVKZS9QbElMZVdSNUtvYVE3dzV5eHVmcGdUalI5VXZ2NmZkTTUzUE45VzV1a1Z3SFFrN2YySGhkbGZUbWFiNWNDWnBZaVBhUnhIUlEwb3RQTW9tUmszdTlhQT09IiwiZXhwIjoxNzYwNjc3NDA1LCJpc3MiOiJNSVNBIiwiYXVkIjoiQU1JU0NSTTIifQ.bUAY1FT6c8VmhUGqfwRobOgRsRjJx4mWtloBaX5iRyI


## 2. Hàm Test Endpoint

In [3]:
def test_endpoint(endpoint_path, access_token, params=None):
    """Hàm chung để test một endpoint với phân trang."""
    if not access_token:
        print('Không có access token, không thể thực hiện request.')
        return
    
    url = f"{BASE_URL}{endpoint_path}"
    headers = {
        'Authorization': f'Bearer {access_token}',
        'Clientid': MISA_CLIENT_ID
    }
    
    # Mặc định lấy trang đầu tiên, 5 bản ghi
    default_params = {'page': 0, 'pageSize': 5}
    if params:
        default_params.update(params)
    
    print(f'--- Đang gọi đến: {url} với params: {default_params} ---')
    
    try:
        response = requests.get(url, headers=headers, params=default_params, timeout=30)
        response.raise_for_status()
        
        result = response.json()
        print(f'Status Code: {response.status_code}')
        print('Response JSON:')
        print(json.dumps(result, indent=2, ensure_ascii=False))
        
        if result.get('success') and isinstance(result.get('data'), list):
            df = pd.DataFrame(result['data'])
            print('Hiển thị dữ liệu dưới dạng DataFrame:')
            display(df)
        else:
            print('Không có dữ liệu hoặc cấu trúc response không đúng.')
            
    except requests.exceptions.RequestException as e:
        print(f'Lỗi request: {e}')

## 3. Test Các Endpoint Cụ Thể

In [4]:
# Endpoint 2: SaleOrders
test_endpoint('/api/v2/SaleOrders', access_token)

--- Đang gọi đến: https://crmconnect.misa.vn/api/v2/SaleOrders với params: {'page': 0, 'pageSize': 5} ---
Status Code: 200
Response JSON:
{
  "success": true,
  "code": 200,
  "data": [
    {
      "id": 3274,
      "sale_order_no": "DH0003274",
      "campaign_name": null,
      "book_date": "2025-07-14T00:00:00.000+07:00",
      "sale_order_amount": 199375000.03,
      "deadline_date": null,
      "quote_name": null,
      "status": "Đã thực hiện",
      "exchange_rate": 1.0,
      "sale_order_name": "Đơn hàng bán cho CÔNG TY TNHH TLT SPORTS",
      "sale_order_date": "2025-07-14T00:00:00.000+07:00",
      "account_name": "CÔNG TY TNHH TLT SPORTS",
      "contact_name": null,
      "due_date": "2025-07-14T00:00:00.000+07:00",
      "delivery_status": "Đã giao hàng",
      "sale_order_type": "Bán mới",
      "opportunity_name": null,
      "revenue_status": "Đã ghi",
      "currency_type": "VND",
      "description": null,
      "balance_receipt_amount": 0.03,
      "pay_status": "Đã 

,id,sale_order_no,campaign_name,book_date,sale_order_amount,deadline_date,quote_name,status,exchange_rate,sale_order_name,...,production_date,production_rejection_reason,produced_quantity_summary,account_code,contact_code,opportunity_code,quote_code,employee_code,is_deleted,sale_order_product_mappings
0,3274,DH0003274,None,2025-07-14T00:00:00.000+07:00,1.993750e+08,None,None,Đã thực hiện,1.0,Đơn hàng bán cho CÔNG TY TNHH TLT SPORTS,...,None,None,0.0,NPPCL01,None,None,None,NV06,False,"[{'id': 12283, 'to_currency_oc': 41666666.7, '..."
1,3256,DH0003256,None,2025-07-13T00:00:00.000+07:00,9.405001e+07,None,None,Đang thực hiện,1.0,Đơn hàng bán cho HỘ KINH DOANH T&D PICKAPOO,...,None,None,0.0,NPP05,None,None,None,NV13,False,"[{'id': 12210, 'to_currency_oc': 50000004.0, '..."
2,3218,DH0003218,None,2025-07-12T00:00:00.000+07:00,7.276500e+07,None,None,Đã thực hiện,1.0,Đơn hàng bán cho CÔNG TY TNHH ĐẦU TƯ THƯƠNG MẠ...,...,None,None,0.0,NPP03,None,None,None,NV09,False,"[{'id': 11964, 'to_currency_oc': 22500000.0, '..."
3,5871,DH101020808,None,2025-10-15T00:00:00.000+07:00,2.700000e+05,None,None,Đang thực hiện,1.0,Đơn hàng tặng cho Lê Đàm Huy trúng giải mini game,...,None,None,0.0,KH20092228,None,None,None,XUANDAO,False,"[{'id': 22059, 'to_currency_oc': 250000.0, 'di..."
4,3236,DH0003236,None,2025-07-11T00:00:00.000+07:00,5.197500e+07,None,None,Đã thực hiện,1.0,Đơn hàng bán cho JP Sport - 01,...,None,None,0.0,NPP19 - 02,None,None,None,DOANTRUNG,False,"[{'id': 12138, 'to_currency_oc': 25000000.02, ..."


In [5]:
# Lấy dữ liệu SaleOrders cho ngày 14 và 15 tháng 10 năm 2025
from datetime import datetime, timezone
import pandas as pd

def get_sale_orders_by_date_range(access_token, start_date, end_date, page_size=100):
    """
    Lấy dữ liệu SaleOrders theo khoảng thời gian
    
    Args:
        access_token: Token xác thực
        start_date: Ngày bắt đầu (YYYY-MM-DD)
        end_date: Ngày kết thúc (YYYY-MM-DD)
        page_size: Số bản ghi mỗi trang
    """
    if not access_token:
        print('Không có access token, không thể thực hiện request.')
        return []
    
    url = f"{BASE_URL}/api/v2/SaleOrders"
    headers = {
        'Authorization': f'Bearer {access_token}',
        'Clientid': MISA_CLIENT_ID,
        'Content-Type': 'application/json'
    }
    
    all_orders = []
    page = 0
    
    print(f"Đang lấy dữ liệu SaleOrders từ {start_date} đến {end_date}...")
    
    while True:
        params = {
            'page': page,
            'pageSize': page_size
        }
        
        try:
            response = requests.get(url, headers=headers, params=params, timeout=30)
            response.raise_for_status()
            
            result = response.json()
            
            if not result.get('success') or not isinstance(result.get('data'), list):
                print(f"Lỗi response: {result}")
                break
                
            orders = result['data']
            if not orders:
                print(f"Không còn dữ liệu ở trang {page}")
                break
            
            # Lọc theo ngày
            filtered_orders = []
            for order in orders:
                sale_order_date = order.get('sale_order_date', '')
                if sale_order_date:
                    # Parse ngày từ format ISO
                    try:
                        order_date = pd.to_datetime(sale_order_date).date()
                        start_dt = datetime.strptime(start_date, '%Y-%m-%d').date()
                        end_dt = datetime.strptime(end_date, '%Y-%m-%d').date()
                        
                        if start_dt <= order_date <= end_dt:
                            filtered_orders.append(order)
                    except Exception as e:
                        print(f"Lỗi parse ngày {sale_order_date}: {e}")
                        continue
            
            all_orders.extend(filtered_orders)
            print(f"Trang {page}: Lấy {len(orders)} orders, lọc được {len(filtered_orders)} orders trong khoảng thời gian")
            
            # Nếu số orders ít hơn page_size, đây là trang cuối
            if len(orders) < page_size:
                break
                
            page += 1
            
        except requests.exceptions.RequestException as e:
            print(f'Lỗi request trang {page}: {e}')
            break
        except Exception as e:
            print(f'Lỗi không xác định trang {page}: {e}')
            break
    
    print(f"Tổng cộng lấy được {len(all_orders)} orders trong khoảng thời gian {start_date} đến {end_date}")
    return all_orders

# Lấy dữ liệu cho ngày 14 và 15 tháng 10 năm 2025
orders_oct_14_15 = get_sale_orders_by_date_range(
    access_token=access_token,
    start_date='2025-10-14',
    end_date='2025-10-15',
    page_size=100
)

print(f"\n=== KẾT QUẢ ===")
print(f"Số lượng orders: {len(orders_oct_14_15)}")

if orders_oct_14_15:
    # Hiển thị thông tin chi tiết các orders
    for i, order in enumerate(orders_oct_14_15[:5]):  # Chỉ hiển thị 5 orders đầu
        print(f"\n--- Order {i+1} ---")
        print(f"Số đơn hàng: {order.get('sale_order_no', 'N/A')}")
        print(f"Tên đơn hàng: {order.get('sale_order_name', 'N/A')}")
        print(f"Ngày đơn hàng: {order.get('sale_order_date', 'N/A')}")
        print(f"Ngày đến hạn: {order.get('due_date', 'N/A')}")
        print(f"Khách hàng: {order.get('account_name', 'N/A')}")
        print(f"Trạng thái: {order.get('status', 'N/A')}")
        print(f"Tổng tiền: {order.get('total_summary', 'N/A')}")
        print(f"Ngày tạo: {order.get('created_date', 'N/A')}")
        print(f"Ngày sửa: {order.get('modified_date', 'N/A')}")
    
    if len(orders_oct_14_15) > 5:
        print(f"\n... và {len(orders_oct_14_15) - 5} orders khác")
    
    # Tạo DataFrame để phân tích
    df_orders = pd.DataFrame(orders_oct_14_15)
    
    # Hiển thị các cột datetime quan trọng
    datetime_cols = ['sale_order_date', 'due_date', 'created_date', 'modified_date']
    available_datetime_cols = [col for col in datetime_cols if col in df_orders.columns]
    
    if available_datetime_cols:
        print(f"\n=== PHÂN TÍCH CÁC TRƯỜNG THỜI GIAN ===")
        for col in available_datetime_cols:
            print(f"\nCột {col}:")
            print(f"  - Số giá trị không null: {df_orders[col].notna().sum()}")
            print(f"  - Các giá trị mẫu:")
            sample_values = df_orders[col].dropna().head(3).tolist()
            for val in sample_values:
                print(f"    + {val}")
else:
    print("Không tìm thấy orders nào trong khoảng thời gian này")


Đang lấy dữ liệu SaleOrders từ 2025-10-14 đến 2025-10-15...
Trang 0: Lấy 100 orders, lọc được 21 orders trong khoảng thời gian
Trang 1: Lấy 100 orders, lọc được 40 orders trong khoảng thời gian
Trang 2: Lấy 100 orders, lọc được 32 orders trong khoảng thời gian
Trang 3: Lấy 100 orders, lọc được 4 orders trong khoảng thời gian
Trang 4: Lấy 100 orders, lọc được 3 orders trong khoảng thời gian
Trang 5: Lấy 100 orders, lọc được 7 orders trong khoảng thời gian
Trang 6: Lấy 100 orders, lọc được 3 orders trong khoảng thời gian
Trang 7: Lấy 100 orders, lọc được 0 orders trong khoảng thời gian
Trang 8: Lấy 100 orders, lọc được 0 orders trong khoảng thời gian
Trang 9: Lấy 100 orders, lọc được 0 orders trong khoảng thời gian
Trang 10: Lấy 100 orders, lọc được 0 orders trong khoảng thời gian
Trang 11: Lấy 100 orders, lọc được 0 orders trong khoảng thời gian
Trang 12: Lấy 100 orders, lọc được 0 orders trong khoảng thời gian
Trang 13: Lấy 100 orders, lọc được 0 orders trong khoảng thời gian
Trang 14:

In [6]:
# Test xử lý datetime để kiểm tra logic timezone
if orders_oct_14_15:
    print("=== TEST XỬ LÝ DATETIME ===")
    
    # Lấy một order mẫu để test
    sample_order = orders_oct_14_15[0]
    
    print(f"Order mẫu: {sample_order.get('sale_order_no', 'N/A')}")
    print(f"Ngày đơn hàng gốc: {sample_order.get('sale_order_date', 'N/A')}")
    print(f"Ngày đến hạn gốc: {sample_order.get('due_date', 'N/A')}")
    
    # Test xử lý datetime theo 2 cách
    sale_order_date_raw = sample_order.get('sale_order_date', '')
    due_date_raw = sample_order.get('due_date', '')
    
    if sale_order_date_raw:
        print(f"\n--- XỬ LÝ SALE_ORDER_DATE ---")
        print(f"Dữ liệu gốc: {sale_order_date_raw}")
        
        # Cách 1: Xử lý SAI (như hiện tại trong sale_orders)
        try:
            wrong_way = pd.to_datetime(sale_order_date_raw, errors="coerce")
            print(f"Cách SAI (không utc=True): {wrong_way}")
            print(f"Timezone: {wrong_way.tzinfo}")
        except Exception as e:
            print(f"Lỗi cách SAI: {e}")
        
        # Cách 2: Xử lý ĐÚNG (như customers/contacts)
        try:
            correct_way_utc = pd.to_datetime(sale_order_date_raw, utc=True, errors="coerce")
            print(f"Cách ĐÚNG (utc=True): {correct_way_utc}")
            print(f"Timezone: {correct_way_utc.tzinfo}")
            
            # Chuyển về timezone-naive
            correct_way_naive = correct_way_utc.tz_convert(None)
            print(f"Sau khi tz_convert(None): {correct_way_naive}")
            print(f"Timezone: {correct_way_naive.tzinfo}")
        except Exception as e:
            print(f"Lỗi cách ĐÚNG: {e}")
    
    if due_date_raw:
        print(f"\n--- XỬ LÝ DUE_DATE ---")
        print(f"Dữ liệu gốc: {due_date_raw}")
        
        # Cách 1: Xử lý SAI
        try:
            wrong_way = pd.to_datetime(due_date_raw, errors="coerce")
            print(f"Cách SAI (không utc=True): {wrong_way}")
            print(f"Timezone: {wrong_way.tzinfo}")
        except Exception as e:
            print(f"Lỗi cách SAI: {e}")
        
        # Cách 2: Xử lý ĐÚNG
        try:
            correct_way_utc = pd.to_datetime(due_date_raw, utc=True, errors="coerce")
            print(f"Cách ĐÚNG (utc=True): {correct_way_utc}")
            print(f"Timezone: {correct_way_utc.tzinfo}")
            
            # Chuyển về timezone-naive
            correct_way_naive = correct_way_utc.tz_convert(None)
            print(f"Sau khi tz_convert(None): {correct_way_naive}")
            print(f"Timezone: {correct_way_naive.tzinfo}")
        except Exception as e:
            print(f"Lỗi cách ĐÚNG: {e}")
    
    print(f"\n=== KẾT LUẬN ===")
    print("1. Dữ liệu gốc từ MISA CRM có timezone +07:00 (giờ Việt Nam)")
    print("2. Xử lý SAI: Giữ nguyên timezone +07:00, hiển thị sai thời gian UTC")
    print("3. Xử lý ĐÚNG: Chuyển từ +07:00 về UTC, hiển thị đúng thời gian UTC")
    print("4. Chênh lệch: 7 giờ (tương đương 1 ngày trong một số trường hợp)")
else:
    print("Không có dữ liệu để test")


=== TEST XỬ LÝ DATETIME ===
Order mẫu: DH101020808
Ngày đơn hàng gốc: 2025-10-15T00:00:00.000+07:00
Ngày đến hạn gốc: 2025-10-15T00:00:00.000+07:00

--- XỬ LÝ SALE_ORDER_DATE ---
Dữ liệu gốc: 2025-10-15T00:00:00.000+07:00
Cách SAI (không utc=True): 2025-10-15 00:00:00+07:00
Timezone: UTC+07:00
Cách ĐÚNG (utc=True): 2025-10-14 17:00:00+00:00
Timezone: UTC
Sau khi tz_convert(None): 2025-10-14 17:00:00
Timezone: None

--- XỬ LÝ DUE_DATE ---
Dữ liệu gốc: 2025-10-15T00:00:00.000+07:00
Cách SAI (không utc=True): 2025-10-15 00:00:00+07:00
Timezone: UTC+07:00
Cách ĐÚNG (utc=True): 2025-10-14 17:00:00+00:00
Timezone: UTC
Sau khi tz_convert(None): 2025-10-14 17:00:00
Timezone: None

=== KẾT LUẬN ===
1. Dữ liệu gốc từ MISA CRM có timezone +07:00 (giờ Việt Nam)
2. Xử lý SAI: Giữ nguyên timezone +07:00, hiển thị sai thời gian UTC
3. Xử lý ĐÚNG: Chuyển từ +07:00 về UTC, hiển thị đúng thời gian UTC
4. Chênh lệch: 7 giờ (tương đương 1 ngày trong một số trường hợp)


In [7]:
# Lấy dữ liệu JSON thô của SaleOrders cho ngày 14 và 15 tháng 10 năm 2025
import json

def get_sale_orders_json_by_date_range(access_token, start_date, end_date, page_size=100):
    """
    Lấy dữ liệu JSON thô của SaleOrders theo khoảng thời gian
    """
    if not access_token:
        print('Không có access token, không thể thực hiện request.')
        return []
    
    url = f"{BASE_URL}/api/v2/SaleOrders"
    headers = {
        'Authorization': f'Bearer {access_token}',
        'Clientid': MISA_CLIENT_ID,
        'Content-Type': 'application/json'
    }
    
    all_orders = []
    page = 0
    
    print(f"Đang lấy dữ liệu JSON SaleOrders từ {start_date} đến {end_date}...")
    
    while True:
        params = {
            'page': page,
            'pageSize': page_size
        }
        
        try:
            response = requests.get(url, headers=headers, params=params, timeout=30)
            response.raise_for_status()
            
            result = response.json()
            
            if not result.get('success') or not isinstance(result.get('data'), list):
                print(f"Lỗi response: {result}")
                break
                
            orders = result['data']
            if not orders:
                print(f"Không còn dữ liệu ở trang {page}")
                break
            
            # Lọc theo ngày (chỉ kiểm tra ngày, không parse datetime)
            filtered_orders = []
            for order in orders:
                sale_order_date = order.get('sale_order_date', '')
                if sale_order_date:
                    # Kiểm tra ngày bằng string matching
                    if start_date in sale_order_date or end_date in sale_order_date:
                        filtered_orders.append(order)
            
            all_orders.extend(filtered_orders)
            print(f"Trang {page}: Lấy {len(orders)} orders, lọc được {len(filtered_orders)} orders trong khoảng thời gian")
            
            # Nếu số orders ít hơn page_size, đây là trang cuối
            if len(orders) < page_size:
                break
                
            page += 1
            
        except requests.exceptions.RequestException as e:
            print(f'Lỗi request trang {page}: {e}')
            break
        except Exception as e:
            print(f'Lỗi không xác định trang {page}: {e}')
            break
    
    print(f"Tổng cộng lấy được {len(all_orders)} orders trong khoảng thời gian {start_date} đến {end_date}")
    return all_orders

# Lấy dữ liệu JSON thô cho ngày 14 và 15 tháng 10 năm 2025
orders_oct_14_15_json = get_sale_orders_json_by_date_range(
    access_token=access_token,
    start_date='2025-10-14',
    end_date='2025-10-15',
    page_size=100
)

print(f"\n=== DỮ LIỆU JSON THÔ ===")
print(f"Số lượng orders: {len(orders_oct_14_15_json)}")

if orders_oct_14_15_json:
    # Hiển thị JSON đầy đủ của từng order
    for i, order in enumerate(orders_oct_14_15_json):
        print(f"\n{'='*50}")
        print(f"ORDER {i+1} - JSON ĐẦY ĐỦ:")
        print(f"{'='*50}")
        print(json.dumps(order, indent=2, ensure_ascii=False))
        
        # Chỉ hiển thị tối đa 3 orders để tránh quá dài
        if i >= 2:
            print(f"\n... và {len(orders_oct_14_15_json) - 3} orders khác")
            break
else:
    print("Không tìm thấy orders nào trong khoảng thời gian này")

Đang lấy dữ liệu JSON SaleOrders từ 2025-10-14 đến 2025-10-15...
Trang 0: Lấy 100 orders, lọc được 21 orders trong khoảng thời gian
Trang 1: Lấy 100 orders, lọc được 39 orders trong khoảng thời gian
Trang 2: Lấy 100 orders, lọc được 33 orders trong khoảng thời gian
Trang 3: Lấy 100 orders, lọc được 4 orders trong khoảng thời gian
Trang 4: Lấy 100 orders, lọc được 3 orders trong khoảng thời gian
Trang 5: Lấy 100 orders, lọc được 7 orders trong khoảng thời gian
Trang 6: Lấy 100 orders, lọc được 3 orders trong khoảng thời gian
Trang 7: Lấy 100 orders, lọc được 0 orders trong khoảng thời gian
Trang 8: Lấy 100 orders, lọc được 0 orders trong khoảng thời gian
Trang 9: Lấy 100 orders, lọc được 0 orders trong khoảng thời gian
Trang 10: Lấy 100 orders, lọc được 0 orders trong khoảng thời gian
Trang 11: Lấy 100 orders, lọc được 0 orders trong khoảng thời gian
Trang 12: Lấy 100 orders, lọc được 0 orders trong khoảng thời gian
Trang 13: Lấy 100 orders, lọc được 0 orders trong khoảng thời gian
Tran